In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexNoLegend"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = VGG16(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 512)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 512)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   44.1s


[CV] ........................... C=0.01, score=0.925490, total=  44.3s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.928429, total=  44.1s
[CV] ........................... C=0.01, score=0.934589, total=  44.2s
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.938976, total=  44.7s
[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.898622, total=  47.3s
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.928571, total=  48.3s
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.943620, total=  54.1s
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.1min


[CV] ........................... C=0.01, score=0.932540, total= 1.1min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.933267, total= 1.1min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.922852, total= 1.2min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.934524, total= 1.2min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.928287, total= 1.2min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.912525, total= 1.2min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.931615, total= 1.2min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.897638, total=  51.1s
[CV] ................... C=1000000000.0, score=0.920744, total=  51.8s
[CV] ................... C=1000000000.0, score=0.918627, total=  51.4s
[CV] ................... C=1000000000.0, score=0.932087, total=  50.1s
[CV] ................... C=1000000000.0, score=0.895153, total=  49.8s
[CV] ................... C=1000000000.0, score=0.915339, total=  45.8s
[CV] .................. C=10000000000.0, score=0.932087, total=  42.9s
[CV] .................. C=10000000000.0, score=0.909449, total=  45.5s
[CV] ................... C=1000000000.0, score=0.917495, total=  55.2s
[CV] ................... C=1000000000.0, score=0.931548, total=  57.9s
[CV] .................. C=10000000000.0, score=0.924805, total=  56.7s
[CV] .................. C=10000000000.0, score=0.922311, total=  51.4s
[CV] ................... C=1000000000.0, score=0.924678, total= 1.0min
[CV] .................. C=10000000000.0, score=0.931548, total=  53.9s


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed:  5.6min remaining:   13.4s


[CV] .................. C=10000000000.0, score=0.914873, total=  55.9s
[CV] .................. C=10000000000.0, score=0.936696, total=  55.0s
[CV] .................. C=10000000000.0, score=0.924678, total=  55.2s
[CV] .................. C=10000000000.0, score=0.917495, total=  54.8s
[CV] .................. C=10000000000.0, score=0.919608, total=  56.7s


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed:  5.6min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([51.99802067, 67.01859722, 60.04262667, 60.50591342, 66.25865457,
        54.88635647, 68.32658255, 57.65130317, 58.95246394, 64.55591428,
        57.11113226, 52.55554128, 52.81038604]),
 'mean_score_time': array([0.01442902, 0.01394472, 0.01168289, 0.01100259, 0.01030731,
        0.01035798, 0.01286602, 0.01099191, 0.01136889, 0.00985703,
        0.01038566, 0.0062633 , 0.00532949]),
 'mean_test_score': array([0.93212313, 0.92472376, 0.92196133, 0.92255328, 0.92127072,
        0.91850829, 0.92284925, 0.92255328, 0.9191989 , 0.918015  ,
        0.92304657, 0.91732439, 0.92334254]),
 'mean_train_score': array([0.98201131, 0.98620927, 0.98499243, 0.98520013, 0.98485165,
        0.97848801, 0.98599057, 0.98560747, 0.97902428, 0.97982042,
        0.98682405, 0.98216485, 0.98583505]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
                    1000000

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.93212, std: 0.00779, params: {'C': 0.01},
 mean: 0.92472, std: 0.01152, params: {'C': 0.1},
 mean: 0.92196, std: 0.01100, params: {'C': 1.0},
 mean: 0.92255, std: 0.00931, params: {'C': 10.0},
 mean: 0.92127, std: 0.01024, params: {'C': 100.0},
 mean: 0.91851, std: 0.01452, params: {'C': 1000.0},
 mean: 0.92285, std: 0.00627, params: {'C': 10000.0},
 mean: 0.92255, std: 0.00924, params: {'C': 100000.0},
 mean: 0.91920, std: 0.01583, params: {'C': 1000000.0},
 mean: 0.91801, std: 0.01275, params: {'C': 10000000.0},
 mean: 0.92305, std: 0.00788, params: {'C': 100000000.0},
 mean: 0.91732, std: 0.01174, params: {'C': 1000000000.0},
 mean: 0.92334, std: 0.00800, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9321
